### Loading the model:

In [1]:
import pickle
import framework as fm
import numpy as np

class Model(fm.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = fm.Conv2d(1, 6, 5, 1, bias = False)
        self.relu1 = fm.ReLu()
        self.maxpool1 = fm.MaxPool2d(2, 6, 2)
        self.conv2 = fm.Conv2d(6, 16, 5, 1, bias = False)
        self.maxpool2 = fm.MaxPool2d(2, 16, 2)
        self.lin1 = fm.Linear(256, 84)
        self.lin2 = fm.Linear(84, 10)


    def forward(self, x):
        x = self.conv1.forward(x)
        print(x.shape)
        x = self.relu1.forward(x)
        print(x.shape)
        x = self.maxpool1.forward(x)
        print(x.shape)
        x = self.conv2.forward(x) 
        print(x.shape)
        x = self.maxpool2.forward(x)
        print(x.shape)
        x = x.reshape((-1, 256))
        print(x.shape)
        x = self.lin1.forward(x) 
        print(x.shape)
        x = self.lin2.forward(x)
        print(x.shape)
        return x


model = Model()

# Load the object back from the file
""" with open('prueba.pkl', 'rb') as file:
    model = pickle.load(file)

print("Object loaded successfully.") """

' with open(\'prueba.pkl\', \'rb\') as file:\n    model = pickle.load(file)\n\nprint("Object loaded successfully.") '

In [2]:
with open('prueba2.pkl', 'rb') as file:
    model = pickle.load(file)

print("Object loaded successfully.")

Object loaded successfully.


### Creation of the API:

In [ ]:
import uvicorn
from fastapi import Request, FastAPI
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/predict")
async def predict(request: Request):
    res = await request.json()
    
    picture = fm.Tensor(np.array(res).reshape((1, 1, 28, 28)))
    pred = model.forward(picture)

    return int(np.argmax(pred.data))

if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()  # Allow asyncio.run to work in a running loop
    uvicorn.run(app, host="127.0.0.1", port=3838)

INFO:     Started server process [11408]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:3838 (Press CTRL+C to quit)


(1, 6, 24, 24)
(1, 6, 24, 24)
(1, 6, 12, 12)
(1, 16, 8, 8)
(1, 16, 4, 4)
(1, 256)
(1, 84)
(1, 10)
INFO:     127.0.0.1:54166 - "POST /predict HTTP/1.1" 200 OK
(1, 6, 24, 24)
(1, 6, 24, 24)
(1, 6, 12, 12)
(1, 16, 8, 8)
(1, 16, 4, 4)
(1, 256)
(1, 84)
(1, 10)
INFO:     127.0.0.1:54174 - "POST /predict HTTP/1.1" 200 OK
(1, 6, 24, 24)
(1, 6, 24, 24)
(1, 6, 12, 12)
(1, 16, 8, 8)
(1, 16, 4, 4)
(1, 256)
(1, 84)
(1, 10)
INFO:     127.0.0.1:54177 - "POST /predict HTTP/1.1" 200 OK
(1, 6, 24, 24)
(1, 6, 24, 24)
(1, 6, 12, 12)
(1, 16, 8, 8)
(1, 16, 4, 4)
(1, 256)
(1, 84)
(1, 10)
INFO:     127.0.0.1:54177 - "POST /predict HTTP/1.1" 200 OK
(1, 6, 24, 24)
(1, 6, 24, 24)
(1, 6, 12, 12)
(1, 16, 8, 8)
(1, 16, 4, 4)
(1, 256)
(1, 84)
(1, 10)
INFO:     127.0.0.1:54177 - "POST /predict HTTP/1.1" 200 OK
(1, 6, 24, 24)
(1, 6, 24, 24)
(1, 6, 12, 12)
(1, 16, 8, 8)
(1, 16, 4, 4)
(1, 256)
(1, 84)
(1, 10)
INFO:     127.0.0.1:54184 - "POST /predict HTTP/1.1" 200 OK
(1, 6, 24, 24)
(1, 6, 24, 24)
(1, 6, 12, 12)
(1, 16,